In [18]:
from __future__ import division
import numpy as np
import csv
import matplotlib.pyplot as plt
import load_data as csv_reader
import matplotlib.patches as mpatches
import seaborn as sns;sns.set()

file_name = '../../Advanced_car_anomaly_detection/Anomaly_Scores/online_version/final_result/scores_0_0_0_0_old.csv'
data = csv_reader.run(file_name)
discrepancy_score = data[:,3]
brightness_score = data[:,2]
discrepancy_score = np.reshape(discrepancy_score, (1,27655)) #27912
brightness_score = np.reshape(brightness_score, (1,27655))

outlier_score = '../../Advanced_car_anomaly_detection/Anomaly_Scores/online_version/final_result/IMU_score_pca_14D_normalized_0.9.csv'
outlier_score = csv_reader.run(outlier_score)
outlier_score = outlier_score[0:27655]
outlier_score = np.reshape(outlier_score, (1,27655)) #27912

traffic_score = '../../Advanced_car_anomaly_detection/Anomaly_Scores/online_version/final_result/traffic_score_27655.csv'
traffic_score = csv_reader.run(traffic_score)
traffic_score = np.reshape(traffic_score, (1,27655)) #27912

# brightness_score = '../Anomaly_Scores/online_version/09072017/score_brightness.csv'
# brightness_score = csv_reader.run(brightness_score)
# brightness_score = np.reshape(brightness_score, (1,27321)) #27912

Rd = discrepancy_score
Rb = brightness_score
Ro = outlier_score
Rt = traffic_score

# tmp = np.random.permutation(27914)
# tmp = tmp[0:593]
# Ro = np.reshape(np.delete(Ro,tmp), (1,27321))
Ro = Ro/max(Ro[0])

print Rd.shape
print Rb.shape
print Ro.shape
print Rt.shape

print max(Rd[0])
print max(Rb[0])
print max(Ro[0])
print max(Rt[0])

# # compute 4 different total scores
# R1 = 0.01* Rd + 0.2*Rb + 0.2*Ro + Rt
# R2 = 0.02* Rd + 0.8*Rb + 0.2*Ro + 0.2*Rt
# R3 = 0.1* Rd + 0.2*Rb + 0.8*Ro + 0.2*Rt
# R4 = 0.2* Rd + 0.2*Rb + 0.2*Ro + 0.5*Rt

# normalize and change shape
Rd = Rd[0]/max(Rd[0])
Rt = Rt[0]
Rb = Rb[0]/max(Rb[0])
Ro = Ro[0]

R1 = 0.25* Rd + 0.25*Rb + 0.25*Ro + 0.25*Rt

(1, 27655)
(1, 27655)
(1, 27655)
(1, 27655)
2.986242854
0.397658003
1.0
1.0


array([[ 0.,  0.,  0., ...,  0.,  0.,  0.]])

In [22]:
# ---------------------- try bayesian filtering and smoothing ------------------------------
from bayes_filter import BayesFilter
%matplotlib osx


delay = 30
observations = Rt
observations[np.where(observations > 0.5)[0]] = 1
observations[np.where(observations <= 0.5)[0]] = 0

BF = BayesFilter(delay, observations[0])

for i in range(2,len(observations)):
    BF.filtering(observations[i])
    if i > delay:
        BF.smoothing()

print BF.prob
print BF.smoothed_prob

index_0 = range(len(observations))
index_1 = range(len(BF.prob))
index_2 = range(len(BF.smoothed_prob))

plt.plot(index_0, observations,'g')
plt.plot(index_1, BF.prob,'r')
plt.plot(index_2, BF.smoothed_prob,'k')
plt.show()


[0.10000000000000001, 0.029408624588716022, 0.0084767595926375679, 0.0026280955041111725, 0.0010213810104309396, 0.00058205742820707687, 0.00046208744261266226, 0.00042933766719400139, 0.00042039838915051677, 0.00041795841508854444, 0.00041729242943451864, 0.00041711065044175097, 0.00041706103438104059, 0.0004170474918199026, 0.00041704379541691774, 0.00041704278649440005, 0.00041704251111191392, 0.00041704243594706004, 0.00041704241543103035, 0.00041704240983123932, 0.00041704240830279257, 0.00041704240788560746, 0.00041704240777173799, 0.0004170424077406576, 0.00041704240773217431, 0.00041704240772985878, 0.0004170424077292268, 0.00041704240772905441, 0.00041704240772900725, 0.0004170424077289944, 0.00041704240772899088, 0.00041704240772898996, 0.00041704240772898969, 0.00041704240772898963, 0.00041704240772898963, 0.00041704240772898963, 0.00041704240772898963, 0.00041704240772898963, 0.00041704240772898963, 0.00041704240772898963, 0.00041704240772898963, 0.00041704240772898963, 0.0

In [13]:
# ------------------------------------------- Draw terrain color bar ----------------------------------------
map_list = np.random.rand(1,27914)
# parking lot
map_list[0, 0:1070] = 0
map_list[0, 27146:27913] = 0
#highway
map_list[0, 4604:7097] = -1
# suburban
map_list[0, 1070:4604] = -2
map_list[0, 7097:8273] = -2
map_list[0, 10787:11078] = -2
map_list[0, 12396:17101] = -2
map_list[0, 17930:20080] = -2
map_list[0, 22174:27146] = -2
# neighborhood
map_list[0, 8273:10787] = 1
map_list[0, 17101:17930] = 1
# off road
map_list[0, 11078:12396] = 2
#downtown
map_list[0, 20080:22174] = 3

In [20]:
R1 = np.reshape(Rt,[1,27655])
smoothed_prob = np.reshape(BF.smoothed_prob, [1, len(BF.smoothed_prob)])

# plot the spectrum 

cmap = sns.color_palette("Set2", 6)

f = plt.figure(figsize=(24,8),dpi=80)
ax = f.add_subplot(311)
ax = sns.heatmap(map_list, ax=ax, cmap=cmap, cbar=False)#="Pastel2"
box = ax.get_position()
ax.set_position([box.x0, box.y0, box.width * 1.1, box.height*0.6])
legend_ax = f.add_axes([0.115, 0.83, 1, .1])
legend_ax.axis('off')

colors = plt.cm.Pastel2(np.linspace(0, 1, 6))

my_dict = ["suburban","highway", "parking lot", "neighborhood", "off road", "downtown"]
patches = [mpatches.Patch(facecolor=c, edgecolor=c) for c in cmap]
legend = legend_ax.legend(patches, my_dict, handlelength=0.8, loc=3, ncol=3, fontsize=20)
for t in legend.get_texts():
    t.set_ha("left")
    
labels = [item.get_text() for item in ax.get_xticklabels()]
labels[0] = '00:00'
for i in range(1,len(labels)-1):   
    labels[i]=''
labels[len(labels)-1] = '44:07'
ax.set_xticklabels(labels, fontsize=22)
ax.set_yticklabels('')
ax.text(-2400, 0.9, 'Terrains: ', style='italic',fontsize=26)

# plot the scores before filtering and smoothing
ax = f.add_subplot(312)
ax = sns.heatmap(R1,ax=ax, cbar=False)
box = ax.get_position()
ax.set_position([box.x0, box.y0, box.width * 1.1, box.height*0.6])
ax.set_xticklabels(labels, fontsize=22)
ax.set_yticklabels('')
ax.text(-1600, 0.7, r'$W_1$:', style='italic',fontsize=26)

# plot the scores after filtering and smoothing

ax = f.add_subplot(313)
ax = sns.heatmap(smoothed_prob,ax=ax, cbar=False)
box = ax.get_position()
ax.set_position([box.x0, box.y0, box.width * 1.1, box.height*0.6])
ax.set_xticklabels(labels, fontsize=22)
ax.set_yticklabels('')
ax.text(-1600, 0.7, r'$W_1$:', style='italic',fontsize=26)



In [21]:
plt.savefig('output.pdf')